In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import glob
import nibabel as nb
import itertools
import json
import sys
import statsmodels.api as sm
from statsmodels.formula.api import ols

from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder


%matplotlib inline

In [87]:
base_dir = '/home/abhijit/Jyotirmay/my_thesis'

In [28]:
flatten = lambda l: [item for sublist in l for item in sublist]

def transform_to_categorical(df, categorical_features_list):
    for f in categorical_features_list:
        dfDummies = pd.get_dummies(df[f], prefix = f)
        df = pd.concat([df, dfDummies], axis=1)
    return df

def rename(df, cols_map):
    df.rename(columns=cols_map, inplace=True)
    return df

def z_score_column_normalise(df, column_list):
    normalised_cols_map = {}
    for column in column_list:
        normalised_cols_map[column] = column+'_normalised'
        df[normalised_cols_map[column]] = (df[column] - df[column].mean())/df[column].std(ddof=0)
    return df, normalised_cols_map


In [3]:
def split_diabetes_state(df):
    df_normal = df[df['diabetes_status']==0]
    df_pre_diabetic = df[df['diabetes_status']==1]
    df_diabetic = df[df['diabetes_status']==2]
    df_normal_affx = df_normal.rename(columns=lambda x: 'normal_'+x)
    df_pre_diabetic_affx = df_pre_diabetic.rename(columns=lambda x: 'pre_diabetic_'+x)
    df_diabetic_affx = df_diabetic.rename(columns=lambda x: 'diabetic_'+x)
    dfs = pd.concat([df_normal_affx, df_pre_diabetic_affx, df_diabetic_affx])
    return dfs

In [4]:
def plot_and_ttest(df, cols):
    dfs[cols].boxplot(rot=45)
    for col_subset in itertools.combinations(cols, 2):
        print(col_subset)
        t,p = stats.ttest_ind(dfs[col_subset[0]].dropna().values, dfs[col_subset[1]].dropna().values)

        print(f'{col_subset[0]} vs {col_subset[1]}')
        print('ttest_score:', t)
        print('p_value:', p)
        print('\n')

In [40]:
def individual_feature_stats(feats, df, target_col):
    p_values = {}
    y = df[target_col].copy()
    y_classes = y.values
    for f in feats:
        try:
            X = pd.get_dummies(df.loc[y.index, [f]], drop_first=True)
            mod = sm.OLS(y_classes, X)
            fii = mod.fit()
            is_significant = True if fii.pvalues.values[0] < 0.05 else False
            p_values[f] = {'coeff': fii.params.values[0], 'p_value':fii.pvalues.values[0],
                           'significant': is_significant}
        except Exception as e:
            print(e)

    return p_values

In [110]:
def group_feature_stats(feats, df, target_col):
    p_values = {}
    y = df[target_col].copy()
    y_classes = y.values
    try:
        X = pd.get_dummies(df.loc[y.index, feats], drop_first=True)
        mod = sm.OLS(y_classes, X)
        fii = mod.fit()
#         df_fii = pd.read_html(fii.summary().tables[1].as_html(),header=0,index_col=0)[0]
        print(fii.pvalues.values[0], fii.params.values)
        
        is_significant_list = [True for p_value in fii.pvalues.values if p_value<0.05]
        print(is_significant_list)
    
        p_values = {'coeff': fii.params.values, 'p_value':fii.pvalues.values, 'significant': is_significant_list}
    except Exception as e:
        print(e)

    return p_values

SyntaxError: invalid syntax (<ipython-input-110-5d60a7e423ef>, line 11)

In [7]:
def normal_group_fit(df, target_col, features_string):
    model = ols(f'{target_col} ~ {features_string}', df).fit()
    return model

def regularised_group_feats(df, target_col, features_string, alpha_col, L1_wt=0.001):
    alpha = df[alpha_col].values
    model = ols(f'{target_col} ~ {features_string}', df).fit_regularized(alpha=alpha, L1_wt=L1_wt,refit=True)
    return model

def normal_mixed_effect_model(df, target_col, features_string, group_col):
    model = ols.mixedlm(f'{target_col} ~ {features_string}', df, groups=df[group_col]).fit()
    return model

def regularised_mixed_effect_model(df, target_col, features_string, group_col, alpha_col, L1_wt=0.001):
    alpha = df[alpha_col].values
    model = ols.mixedlm(f'{target_col} ~ {features_string}', df, groups=df[group_col]).fit_regularized(alpha=alpha, L1_wt=L1_wt,refit=True)
    return model

def anova_test(ols_model):
    anova_stats = sm.stats.anova_lm(ols_model)
    return anova_stats

In [8]:
def gathering_p_values():
    pass

In [105]:
def df_from_nested_dicts(dicts):
    df = pd.concat({k+'_'+kk: pd.concat({kk:pd.DataFrame(vv).T}, axis=0) for k, v in dicts.items() for kk, vv in v.items()}, axis=0)
    return df

In [34]:
smoking_feats = ['smoker_former', 'smoker_irregular', 'smoker_non-smoker', 'smoker_regular', 'smoking-packages']
bmi_feats = ['bmi-who_normal', 'bmi-who_obesity class I', 'bmi-who_obesity class II', 'bmi-who_obesity class III',
            'bmi-numeric', 'bmi-who_pre-obisety']
blood_pressure_feats = ['blood-pressure-diastolic', 'blood-pressure-systolic']
cholesterol_feats = ['cholesterol-hdl', 'cholesterol-ldl', 'cholesterol-total']
mri_feats = ['mri-liver-fat-artifacts', 'mri-liver-fat-lobus-dexter', 
             'mri-liver-fat-lobus-sinister', 'mri-liver-fat-portal-vein']
alcohol_feats = ['alcohol-g/day']
hbalc_feats = ['hba1c-mmol/mol', 'hba1c-percentage']
medication_feats = ['meds-antidiabetic', 'meds-antihypertensive', 'meds-incretin-mimetics', 'meds-insulin-therapy',
                    'meds-lipoprotein-lowering', 'meds-oral-antidiabetic']
triglyceride = ['triglyceride']
hypertension = ['hypertension']
basic_feats = ['age', 'height', 'sex', 'weight' ]
vols_feat = ['seg_liver', 'seg_spleen']

feats_from_paper_for_group_test = [['age', 'sex_0', 'sex_1', 'bmi_numeric'],
             ['diabetes_status_0', 'diabetes_status_1', 'diabetes_status_2'], ['hypertension'], ['triglyceride'],
             ['cholesterol-hdl', 'cholesterol-ldl'],
             ['mri-liver-fat-artifacts', 'mri-liver-fat-lobus-dexter', 
              'mri-liver-fat-lobus-sinister', 'mri-liver-fat-portal-vein'],
             ['meds-lipoprotein-lowering'],
             ['smoker_former', 'smoker_non-smoker', 'smoker_regular']]

feats_from_paper_for_individual_test = [['age', 'sex_0', 'sex_1', 'bmi_numeric'],
             ['diabetes_status_0', 'diabetes_status_1', 'diabetes_status_2'], ['hypertension'], ['triglyceride'],
             ['blood-pressure-diastolic', 'blood-pressure-systolic'],
             ['cholesterol-hdl', 'cholesterol-ldl', 'cholesterol-total'],
             ['mri-liver-fat-artifacts', 'mri-liver-fat-lobus-dexter', 
              'mri-liver-fat-lobus-sinister', 'mri-liver-fat-portal-vein'],
             ['meds-lipoprotein-lowering', 'meds-antihypertensive'],
             ['smoker_former', 'smoker_non-smoker', 'smoker_regular'], ['alcohol-g/day']]

In [35]:
model_merged_feats_path = [
    {'full_bayesian': './projects/full_bayesian/reports/full_bayesian_KORA_v2/KORA/10_1571866968.4002764_concat_report_final.csv'},
    {'MC_dropout_quicknat': './projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_KORA_v2/KORA/10_1572006141.7793334_concat_report_final.csv'}, 
    {'probabilistic_quicknat': './projects/probabilistic_quicknat/reports/probabilistic_quicknat_KORA_v2/KORA/10_1571996796.7963011_concat_report_final.csv'}, 
    {'hierarchical_quicknat': './projects/hierarchical_quicknat/reports/hierarchical_quicknat_KORA_v2/KORA/10_1571905560.9377904_concat_report_final.csv'}
]
all_dataset_merged_feats_path = [
    {'all_KORA_processed_False': '/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/KORA/all_processed_False_concat_report_final.csv'}, 
    {'all_KORA_processed_True': '/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/KORA/all_processed_True_concat_report_final.csv'}
]

test_dataset_merged_feats_path = [
    {'test_KORA_processed_False': '/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/KORA/test_processed_False_concat_report_final.csv'}, 
    {'test_KORA_processed_True': '/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/KORA/test_processed_True_concat_report_final.csv'}
]

fb = 'full_bayesian'
mc = 'MC_dropout_quicknat'
pq = 'probabilistic_quicknat'
hq = 'hierarchical_quicknat'
af = 'all_KORA_processed_False'
at = 'all_KORA_processed_True'
tf = 'test_KORA_processed_False'
tt = 'test_KORA_processed_True'

In [36]:
model_merged_feats_path_combined = {key:val for d in model_merged_feats_path for key,val in d.items()}

# Individual feats stats test

In [112]:
feats = flatten(feats_from_paper_for_individual_test)

dicts = {}
for key, value in model_merged_feats_path_combined.items():
    df = pd.read_csv(value)
    df = rename(df, {'bmi-numeric':'bmi_numeric'})
    df = transform_to_categorical(df, ['diabetes_status', 'sex'])
    df, normalised_cols = z_score_column_normalise(df, ['seg_spleen', 'seg_liver'])
    df = df.fillna(0)

    dicts[key] = {}
    
    target_col = 'seg_spleen_normalised'
    p_value_dict = individual_feature_stats(feats, df, target_col)
    dicts[key][target_col] = p_value_dict
    
    target_col = 'seg_liver_normalised'
    p_value_dict = individual_feature_stats(feats, df, target_col)
    dicts[key][target_col] = p_value_dict

p_value_df = df_from_nested_dicts(dicts)
p_value_df

coeff  \
full_bayesian_seg_spleen_normalised        seg_spleen_normalised age                   -0.000582568   
                                                                 sex_0                    -0.402445   
                                                                 sex_1                     0.274193   
                                                                 bmi_numeric             0.00191266   
                                                                 diabetes_status_0       -0.0780769   
...                                                                                             ...   
hierarchical_quicknat_seg_liver_normalised seg_liver_normalised  meds-antihypertensive     0.426677   
                                                                 smoker_former            0.0801909   
                                                                 smoker_non-smoker        -0.102257   
                                                                 smoker_regular             0.05443   
                                                                 alcohol-g/day           0.00335979   

                                                                                           p_value  \
full_bayesian_seg_spleen_normalised        seg_spleen_normalised age                      0.686873   
                                                                 sex_0                  0.00134136   
                                                                 sex_1                  0.00847271   
                                                                 bmi_numeric               0.50035   
                                                                 diabetes_status_0         0.43674   
...                                                                                            ...   
hierarchical_quicknat_seg_liver_normalised seg_liver_normalised  meds-antihypertensive  0.00729456   
                                                                 smoker_former            0.510221   
                                                                 smoker_non-smoker        0.462693   
                                                                 smoker_regular           0.790722   
                                                                 alcohol-g/day            0.223994   

                                                                                       significant  
full_bayesian_seg_spleen_normalised        seg_spleen_normalised age                         False  
                                                                 sex_0                        True  
                                                                 sex_1                        True  
                                                                 bmi_numeric                 False  
                                                                 diabetes_status_0           False  
...                                                                                            ...  
hierarchical_quicknat_seg_liver_normalised seg_liver_normalised  meds-antihypertensive        True  
                                                                 smoker_former               False  
                                                                 smoker_non-smoker           False  
                                                                 smoker_regular              False  
                                                                 alcohol-g/day               False  

[192 rows x 3 columns]

In [113]:
# p_value_df = pd.concat({k+'_'+kk:pd.DataFrame(vv).T for k, v in dicts.items() for kk, vv in v.items()}, axis=1)
# p_value_df = pd.concat({k+'_'+kk: pd.concat({kk:pd.DataFrame(vv).T}, axis=0) for k, v in dicts.items() for kk, vv in v.items()}, axis=0)

# p_value_df.to_csv(f'{base_dir}/statistical_analysis_report/individual_feats_stats_test_report.csv')

p_value_df[p_value_df['significant']==True]

coeff  \
full_bayesian_seg_spleen_normalised          seg_spleen_normalised sex_0                          -0.402445   
                                                                   sex_1                           0.274193   
                                                                   mri-liver-fat-artifacts         -1.40657   
                                                                   mri-liver-fat-portal-vein       0.016092   
full_bayesian_seg_liver_normalised           seg_liver_normalised  sex_0                          -0.335198   
                                                                   sex_1                           0.228377   
                                                                   diabetes_status_1               0.341617   
                                                                   hypertension                    0.386793   
                                                                   triglyceride                  0.00117223   
                                                                   meds-antihypertensive           0.340433   
MC_dropout_quicknat_seg_spleen_normalised    seg_spleen_normalised sex_0                          -0.392069   
                                                                   sex_1                           0.267124   
MC_dropout_quicknat_seg_liver_normalised     seg_liver_normalised  sex_0                          -0.552665   
                                                                   sex_1                           0.376541   
                                                                   diabetes_status_0               -0.27021   
                                                                   diabetes_status_2               0.891634   
                                                                   hypertension                    0.551368   
                                                                   triglyceride                  0.00160622   
                                                                   cholesterol-hdl              -0.00250426   
                                                                   mri-liver-fat-lobus-dexter     0.0310688   
                                                                   mri-liver-fat-lobus-sinister   0.0350286   
                                                                   mri-liver-fat-portal-vein      0.0298633   
                                                                   meds-antihypertensive           0.443168   
probabilistic_quicknat_seg_spleen_normalised seg_spleen_normalised sex_0                          -0.410996   
                                                                   sex_1                           0.280019   
                                                                   mri-liver-fat-portal-vein      0.0151443   
probabilistic_quicknat_seg_liver_normalised  seg_liver_normalised  sex_0                          -0.569146   
                                                                   sex_1                            0.38777   
                                                                   diabetes_status_0              -0.259498   
                                                                   diabetes_status_2               0.806143   
                                                                   hypertension                    0.529487   
                                                                   triglyceride                  0.00155036   
                                                                   cholesterol-hdl              -0.00251202   
                                                                   mri-liver-fat-lobus-dexter     0.0304921   
                                                                   mri-liver-fat-lobus-sinister   0.0344948   
                                                                   mri-liver-fat-portal-vein      0.029645

# Group feats stats test

In [108]:
feats = flatten(feats_from_paper_for_group_test)

dicts = {}
for key, value in model_merged_feats_path_combined.items():
    df = pd.read_csv(value)
    df = rename(df, {'bmi-numeric':'bmi_numeric'})
    df = transform_to_categorical(df, ['diabetes_status', 'sex'])
    df, normalised_cols = z_score_column_normalise(df, ['seg_spleen', 'seg_liver'])
    df = df.fillna(0)

    dicts[key] = {}
    
    target_col = 'seg_spleen_normalised'
    p_value_dict = group_feature_stats(feats, df, target_col)
    dicts[key][target_col] = p_value_dict
    
    target_col = 'seg_liver_normalised'
    p_value_dict = group_feature_stats(feats, df, target_col)
    dicts[key][target_col] = p_value_dict

p_value_df = df_from_nested_dicts(dicts)
p_value_df.columns = feats
p_value_df

0.06478539263263022 [-1.65784017e-02  4.05193349e-01  9.51419520e-01  4.21693646e-02
  4.25072106e-01  3.42862074e-01  5.88678689e-01 -1.30801135e-01
 -1.05422982e-03 -8.61153893e-03 -4.58729550e-03 -1.33446849e+00
 -2.29269319e-02 -7.29220359e-03  3.73864890e-02 -4.89029146e-01
  1.28225600e-01 -1.66882679e-01 -3.26502062e-01]
0.000625486075252123 [-0.02798549 -0.10396447  0.16029238  0.08356735 -0.07950574  0.14040237
 -0.00456873  0.28702987  0.00177969 -0.00732076 -0.00484471 -0.22762844
 -0.01689144 -0.02319503  0.00853891 -0.18815962  0.24427161  0.19154552
  0.22229574]
0.20094663311966174 [-0.01072171 -0.35844494  0.18220209  0.08279846 -0.08588812 -0.23183878
  0.14148406  0.01790393 -0.00075857 -0.00767612 -0.00659823  0.19445885
 -0.02840315 -0.00904144  0.02625026 -0.70304924  0.19135573 -0.03365804
 -0.19894131]
1.6075059994989203e-06 [-0.02983311 -0.06126131  0.37171266  0.06678799  0.06497023  0.00520236
  0.24027877  0.27179344  0.00206401 -0.00595323 -0.0061257  -0.070

age  \
full_bayesian_seg_spleen_normalised          seg_spleen_normalised coeff   -0.016578   
                                                                   p_value  0.064785   
full_bayesian_seg_liver_normalised           seg_liver_normalised  coeff   -0.027985   
                                                                   p_value  0.000625   
MC_dropout_quicknat_seg_spleen_normalised    seg_spleen_normalised coeff   -0.010722   
                                                                   p_value  0.200947   
MC_dropout_quicknat_seg_liver_normalised     seg_liver_normalised  coeff   -0.029833   
                                                                   p_value  0.000002   
probabilistic_quicknat_seg_spleen_normalised seg_spleen_normalised coeff   -0.014154   
                                                                   p_value  0.105251   
probabilistic_quicknat_seg_liver_normalised  seg_liver_normalised  coeff   -0.028256   
                                                                   p_value  0.000015   
hierarchical_quicknat_seg_spleen_normalised  seg_spleen_normalised coeff   -0.013897   
                                                                   p_value  0.117435   
hierarchical_quicknat_seg_liver_normalised   seg_liver_normalised  coeff   -0.030220   
                                                                   p_value  0.000002   

                                                                               sex_0  \
full_bayesian_seg_spleen_normalised          seg_spleen_normalised coeff    0.405193   
                                                                   p_value  0.497426   
full_bayesian_seg_liver_normalised           seg_liver_normalised  coeff   -0.103964   
                                                                   p_value  0.846050   
MC_dropout_quicknat_seg_spleen_normalised    seg_spleen_normalised coeff   -0.358445   
                                                                   p_value  0.521732   
MC_dropout_quicknat_seg_liver_normalised     seg_liver_normalised  coeff   -0.061261   
                                                                   p_value  0.877763   
probabilistic_quicknat_seg_spleen_normalised seg_spleen_normalised coeff    0.196210   
                                                                   p_value  0.735885   
probabilistic_quicknat_seg_liver_normalised  seg_liver_normalised  coeff    0.326511   
                                                                   p_value  0.439862   
hierarchical_quicknat_seg_spleen_normalised  seg_spleen_normalised coeff    0.040090   
                                                                   p_value  0.945916   
hierarchical_quicknat_seg_liver_normalised   seg_liver_normalised  coeff    0.006024   
                                                                   p_value  0.988150   

                                                                               sex_1  \
full_bayesian_seg_spleen_normalised          seg_spleen_normalised coeff    0.951420   
                                                                   p_value  0.097217   
full_bayesian_seg_liver_normalised           seg_liver_normalised  coeff    0.160292   
                                                                   p_value  0.754349   
MC_dropout_quicknat_seg_spleen_normalised    seg_spleen_normalised coeff    0.182202   
                                                                   p_value  0.733380   
MC_dropout_quicknat_seg_liver_normalised     seg_liver_normalised  coeff    0.371713   
                                                                   p_value  0.330094   
probabilistic_quicknat_seg_spleen_normalised seg_spleen_normalised coeff    0.746528   
                                                                   p_value  0.181078   
probabilistic_quicknat_seg_liver_normalised  seg_liver_normalised  coeff    0.777310   
                               

In [109]:
p_value_df.T

full_bayesian_seg_spleen_normalised            \
                                           seg_spleen_normalised             
                                                           coeff   p_value   
age                                                    -0.016578  0.064785   
sex_0                                                   0.405193  0.497426   
sex_1                                                   0.951420  0.097217   
bmi_numeric                                             0.042169  0.026244   
diabetes_status_0                                       0.425072  0.255057   
diabetes_status_1                                       0.342862  0.431028   
diabetes_status_2                                       0.588679  0.180355   
hypertension                                           -0.130801  0.487402   
triglyceride                                           -0.001054  0.266613   
cholesterol-hdl                                        -0.008612  0.127042   
cholesterol-ldl                                        -0.004587  0.082676   
mri-liver-fat-artifacts                                -1.334468  0.035939   
mri-liver-fat-lobus-dexter                             -0.022927  0.475367   
mri-liver-fat-lobus-sinister                           -0.007292  0.806464   
mri-liver-fat-portal-vein                               0.037386  0.070226   
meds-lipoprotein-lowering                              -0.489029  0.090008   
smoker_former                                           0.128226  0.687214   
smoker_non-smoker                                      -0.166883  0.604564   
smoker_regular                                         -0.326502  0.344252   

                             full_bayesian_seg_liver_normalised            \
                                           seg_liver_normalised             
                                                          coeff   p_value   
age                                                   -0.027985  0.000625   
sex_0                                                 -0.103964  0.846050   
sex_1                                                  0.160292  0.754349   
bmi_numeric                                            0.083567  0.000002   
diabetes_status_0                                     -0.079506  0.812060   
diabetes_status_1                                      0.140402  0.719106   
diabetes_status_2                                     -0.004569  0.990725   
hypertension                                           0.287030  0.090904   
triglyceride                                           0.001780  0.037727   
cholesterol-hdl                                       -0.007321  0.148143   
cholesterol-ldl                                       -0.004845  0.041553   
mri-liver-fat-artifacts                               -0.227628  0.687775   
mri-liver-fat-lobus-dexter                            -0.016891  0.557751   
mri-liver-fat-lobus-sinister                          -0.023195  0.385804   
mri-liver-fat-portal-vein                              0.008539  0.643066   
meds-lipoprotein-lowering                             -0.188160  0.465348   
smoker_former                                          0.244272  0.393194   
smoker_non-smoker                                      0.191546  0.507885   
smoker_regular                                         0.222296  0.472712   

                             MC_dropout_quicknat_seg_spleen_normalised  \
                                                 seg_spleen_normalised   
                                                                 coeff   
age                                                          -0.010722   
sex_0                                                        -0.358445   
sex_1                                                         0.182202   
bmi_numeric                                                   0.082798   
diabetes_status_0                                            -0.085888   
diabetes_status_1        

# Regularised group feat stats test

# Anova test

# Mixed Effect Model test